In [1]:
#!/usr/bin/env python
import numpy as np
import math as m
import pylab as pl
from matplotlib import colors
from matplotlib import gridspec
import matplotlib.ticker as ticker
from matplotlib.ticker import ScalarFormatter
from colormaps import spectral
from matplotlib.colors import ColorConverter

import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

# Font properties
# Note: TNR required for bolding due to font manager bug
# Math font for compact, times-like scientific notation
# Exit out of notebook before changing
times_font=True
if times_font:
    font_bold = fm.FontProperties(family='Times New Roman', style='normal', size=14, weight='bold', stretch='normal')
    font      = fm.FontProperties(family='Times', style='normal', size=14, weight='normal', stretch='normal')
    font_name ='Times'
    plt.rcParams["mathtext.fontset"] = "stix"#"stixsans"
else:
    font_name ='Dejavu Sans'
    font_bold = fm.FontProperties(family='Dejavu Sans', style='normal', size=14, weight='bold', stretch='normal')
    font      = fm.FontProperties(family='Dejavu Sans', style='normal', size=14, weight='normal', stretch='normal')


In [61]:
# noise plot

ColorConverter.colors['e']=(94/256, 79/256, 162/256)
ColorConverter.colors['g']=(50/256, 136/256, 189/256)
ColorConverter.colors['h']=(102/256, 194/256, 165/256)
ColorConverter.colors['i']=(253/256, 174/256, 97/256)
ColorConverter.colors['j']=(219/256, 55/256, 82/256)

# Data file names
data_dir = '/Users/z3543600/tmp/noise/'
data_base = ['gridded_noise_gridded_uvf_XX__even_odd_joint_noimgclip_dirty_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_now.txt',\
            'gridded_noise_gridded_uvf_XX__even_odd_joint_noimgclip_dirty_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_wstack.txt',\
            'gridded_noise_gridded_uvf_XX__even_odd_joint_noimgclip_dirty_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_awproj.txt',\
            'Combined_obs_noise_cubeXX__even_odd_joint_noimgclip_res_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_now.txt',\
            'Combined_obs_noise_cubeXX__even_odd_joint_noimgclip_res_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_wstack.txt',\
            'Combined_obs_noise_cubeXX__even_odd_joint_noimgclip_res_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_awproj.txt',\
            'Combined_obs_noise_cubeXX__even_odd_joint_noimgclip_res_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_awproj.txt']
data_col = [[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1]]

# Plot styles per data set
colors=['j','i','g','j','i','g','black']
linestyles=['-','-','-','--','--','--','-']

fontsize=16
ticksize=16
legendsize=16

# Initialise the plot
pl.figure(figsize=(8,6))

for file_i, file_name in enumerate(data_base):
    #
    # Load data and select
    data = np.loadtxt(data_dir+file_name,skiprows=1)
    k_edges = data[:,(data_col[file_i])[0]] # Lower and upper edge of bin, (h Mpc^-1)
    power = data[1:,(data_col[file_i])[1]] # Observed power in each bin, (mK^2 h^-3 Mpc^3), first entry is a placeholder
        
    # Construct k center from the edges
    n_k_edges = len(k_edges)
    k_center = (k_edges[1:n_k_edges]+k_edges[0:n_k_edges-1])/2.
        
    # Construct power in mK^2 units
    delta = (power)#*(k_center**3.)/(2*np.pi**2)
        
    # Create histogram style data set by repeating the data 
    # Duplicate the bin edges so each bin has left and right data point
    # Duplicate the power so each bin edge has a measurement
    k_edges_hist  = np.repeat(k_edges,2)[1:-1]
    delta_hist  = np.repeat(delta,2)
    
    if file_i == 6:
        delta_hist[:] = 8.923

    delta_hist = delta_hist / 8.923
    # Plot the data on the initialised figure
    pl.plot(k_edges_hist, delta_hist, color=colors[file_i],linewidth=2, linestyle=linestyles[file_i])
        
# Final plotting settings and save

pl.loglog()
pl.tick_params(direction='in', color='k', which='both', top=True, bottom=True, left=True, right=True, labelsize=ticksize)
pl.xlim(1e-2,2)
pl.ylim(1e-1, 1e1)
#pl.ylabel(r'$\Delta^2$ (mK$^2$)', fontsize=fontsize, family=font_name)
pl.ylabel(r'Ratio from expected (h$^{-3}$ mK$^2$ Mpc$^{3}$)', fontsize=fontsize, family=font_name)
pl.xlabel(r'$k$ (h Mpc$^{-1}$)', fontsize=fontsize, family=font_name)
pl.title('Noise simulations, 5 observations',fontsize=fontsize, family=font_name)

# Tick formatting
#pl.yticks([1e-0,1e3,1e6],fontname = font_name)
pl.xticks(fontname = font_name)
pl.yticks(fontname = font_name)
a=pl.gca()
a.yaxis.set_minor_locator(ticker.LogLocator(base=10, numticks=20))
a.yaxis.set_minor_formatter(ticker.NullFormatter())

# Legend
#f1_label = ['EoR z=7 theory','WODEN vis through FHD/eppsilon', 'FHD simulation']
f1_label = ['No w-correction, HEALPix','W-stacking, HEALPix','AW-projection, HEALPix','No w-correction, gridded','W-stacking, gridded','AW-projection, gridded']
patches = [ pl.plot([],[], mec=None, color=colors[type_i], linestyle=linestyles[type_i],
            label="{:s}".format(f1_label[type_i]) )[0]  for type_i in range(len(f1_label)) ]
legend1 = pl.legend(handles=patches, bbox_to_anchor=(0.75, 0.21),loc='center', ncol=1, frameon=False,
                    fontsize=legendsize,title_fontsize=legendsize,prop=font)
#legend1.set_title(title="Derivative matched\nto control cut percent", prop=font_bold)
legend1._legend_box.align='left'
a.add_artist(legend1)


pl.savefig(data_dir+'noise.png', bbox_inches='tight', dpi=128)
pl.close('all')

In [60]:
# EoR and foregrounds plot

ColorConverter.colors['e']=(94/256, 79/256, 162/256)
ColorConverter.colors['g']=(50/256, 136/256, 189/256)
ColorConverter.colors['h']=(102/256, 194/256, 165/256)
ColorConverter.colors['i']=(253/256, 174/256, 97/256)
ColorConverter.colors['j']=(219/256, 55/256, 82/256)

# Data file names
data_dir = '/Users/z3543600/tmp/EoRfore/'
data_base = ['gridded_EoR_gridded_uvf_XX__even_odd_joint_noimgclip_dirty_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_now.txt',\
            'gridded_EoR_gridded_uvf_XX__even_odd_joint_noimgclip_dirty_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_wstack.txt',\
            'gridded_EoR_gridded_uvf_XX__even_odd_joint_noimgclip_dirty_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_awproj.txt',\
            'Combined_obs_EoR_cubeXX__even_odd_joint_noimgclip_res_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_now.txt',\
            'Combined_obs_EoR_cubeXX__even_odd_joint_noimgclip_res_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_wstack.txt',\
            'Combined_obs_EoR_cubeXX__even_odd_joint_noimgclip_res_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_awproj.txt']
data_col = [[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1]]

# Plot styles per data set
colors=['j','i','g','j','i','g','black']
linestyles=['-','-','-','--','--','--','-']

fontsize=16
ticksize=16
legendsize=16

# Initialise the plot
pl.figure(figsize=(8,6))

for file_i, file_name in enumerate(data_base):
    #
    # Load data and select
    data = np.loadtxt(data_dir+file_name,skiprows=1)
    k_edges = data[:,(data_col[file_i])[0]] # Lower and upper edge of bin, (h Mpc^-1)
    power = data[1:,(data_col[file_i])[1]] # Observed power in each bin, (mK^2 h^-3 Mpc^3), first entry is a placeholder
        
    # Construct k center from the edges
    n_k_edges = len(k_edges)
    k_center = (k_edges[1:n_k_edges]+k_edges[0:n_k_edges-1])/2.
        
    # Construct power in mK^2 units
    delta = (power)*(k_center**3.)/(2*np.pi**2)
        
    # Create histogram style data set by repeating the data 
    # Duplicate the bin edges so each bin has left and right data point
    # Duplicate the power so each bin edge has a measurement
    k_edges_hist  = np.repeat(k_edges,2)[1:-1]
    delta_hist  = np.repeat(delta,2)

    # Plot the data on the initialised figure
    pl.plot(k_edges_hist, delta_hist, color=colors[file_i],linewidth=2, linestyle=linestyles[file_i])
        
# Final plotting settings and save

pl.loglog()
pl.tick_params(direction='in', color='k', which='both', top=True, bottom=True, left=True, right=True, labelsize=ticksize)
pl.xlim(1e-2,2)
pl.ylim(1e2, 1e9)
#pl.ylabel(r'$\Delta^2$ (mK$^2$)', fontsize=fontsize, family=font_name)
pl.ylabel(r'$\Delta^2$ (mK$^2$)', fontsize=fontsize, family=font_name)
pl.xlabel(r'$k$ (h Mpc$^{-1}$)', fontsize=fontsize, family=font_name)
pl.title('EoR and foreground simulations, coarse FEE beam, 15 observations',fontsize=fontsize, family=font_name)

# Tick formatting
pl.yticks([1e3,1e5,1e7,1e9],fontname = font_name)
pl.xticks(fontname = font_name)
a=pl.gca()
a.yaxis.set_minor_locator(ticker.LogLocator(base=10, numticks=20))
a.yaxis.set_minor_formatter(ticker.NullFormatter())

# Legend
#f1_label = ['EoR z=7 theory','WODEN vis through FHD/eppsilon', 'FHD simulation']
f1_label = ['No w-correction, HEALPix','W-stacking, HEALPix','AW-projection, HEALPix','No w-correction, gridded','W-stacking, gridded','AW-projection, gridded']
patches = [ pl.plot([],[], mec=None, color=colors[type_i], linestyle=linestyles[type_i],
            label="{:s}".format(f1_label[type_i]) )[0]  for type_i in range(len(f1_label)) ]
legend1 = pl.legend(handles=patches, bbox_to_anchor=(0.25, 0.21),loc='center', ncol=1, frameon=False,
                    fontsize=legendsize,title_fontsize=legendsize,prop=font)
#legend1.set_title(title="Derivative matched\nto control cut percent", prop=font_bold)
legend1._legend_box.align='left'
a.add_artist(legend1)


pl.savefig(data_dir+'EoRfore.png', bbox_inches='tight', dpi=128)
pl.close('all')

In [50]:
#EoR plotting only

ColorConverter.colors['e']=(94/256, 79/256, 162/256)
ColorConverter.colors['g']=(50/256, 136/256, 189/256)
ColorConverter.colors['h']=(102/256, 194/256, 165/256)
ColorConverter.colors['i']=(253/256, 174/256, 97/256)
ColorConverter.colors['j']=(219/256, 55/256, 82/256)

# Data file names
data_dir = '/Users/z3543600/tmp/EoR/'
data_base = ['gridded_EoR_gridded_uvf_XX__even_odd_joint_noimgclip_dirty_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_now.txt',\
            'gridded_EoR_gridded_uvf_XX__even_odd_joint_noimgclip_dirty_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_wstack.txt',\
            'gridded_EoR_gridded_uvf_XX__even_odd_joint_noimgclip_dirty_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_awproj.txt',\
            'Combined_obs_EoR_cubeXX__even_odd_joint_noimgclip_res_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_now.txt',\
            'Combined_obs_EoR_cubeXX__even_odd_joint_noimgclip_res_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_wstack.txt',\
            'Combined_obs_EoR_cubeXX__even_odd_joint_noimgclip_res_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_awproj.txt']
data_col = [[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1]]

# Plot styles per data set
colors=['j','i','g','j','i','g','black']
linestyles=['-','-','-','--','--','--','-']

fontsize=16
ticksize=16
legendsize=16

# Initialise the plot
#pl.figure(figsize=(8,6))


fig, (ax_main, ax_diff) = plt.subplots(
    2, 1, sharex=True, figsize=(8, 7),
    gridspec_kw={'height_ratios':[3,1], 'hspace':0.05}
)


brad_data = np.loadtxt(data_dir+'delTps_estimate_DIM6400_NSlices420_7500Mpc_lighttravel.txt')
   
ax_main.plot(brad_data[:,0]/0.68, brad_data[:,1],
        linestyle='-', color='black', label='Expected')

for file_i, file_name in enumerate(data_base):
    #
    # Load data and select
    data = np.loadtxt(data_dir+file_name,skiprows=1)
    k_edges = data[:,(data_col[file_i])[0]] # Lower and upper edge of bin, (h Mpc^-1)
    power = data[1:,(data_col[file_i])[1]] # Observed power in each bin, (mK^2 h^-3 Mpc^3), first entry is a placeholder
        
    # Construct k center from the edges
    n_k_edges = len(k_edges)
    k_center = (k_edges[1:n_k_edges]+k_edges[0:n_k_edges-1])/2.
        
    # Construct power in mK^2 units
    delta = (power)*(k_center**3.)/(2*np.pi**2)
    #k_edges = k_edges*1.4
        
    # Create histogram style data set by repeating the data 
    # Duplicate the bin edges so each bin has left and right data point
    # Duplicate the power so each bin edge has a measurement
    k_edges_hist  = np.repeat(k_edges,2)[1:-1]
    delta_hist  = np.repeat(delta,2)
        
    # Plot the data on the initialised figure
    ax_main.plot(k_edges_hist, delta_hist, color=colors[file_i], linestyle=linestyles[file_i],linewidth=2)

    # ----------------------------
    # Bottom panel: residuals
    # Interpolate theory to the same k as the plotted histogram points.
    # Use np.interp which expects sorted x; your k arrays are increasing in log scale already.
    # ----------------------------

    # Define bin-center theory (or theory per bin)
    #    You have continuous theory (k_theory, delta_theory). To compare fairly per bin:
    k_lo = k_edges[:-1]
    k_hi = k_edges[1:]
    k_ctr = 0.5*(k_lo + k_hi)
    
    # Interpolate theory at bin centers
    theory_ctr = np.interp(k_ctr, brad_data[:,0]/0.68, brad_data[:,1], left=np.nan, right=np.nan)
    
    ratio_per_bin = (delta - theory_ctr)/theory_ctr
    
    # Plot as true histogram using bin edges
    valid = np.isfinite(ratio_per_bin)
    ax_diff.stairs(ratio_per_bin[valid], k_edges[np.r_[valid, False] | np.r_[False, valid]],  # keep aligned edges
                   color=colors[file_i], linestyle=linestyles[file_i], linewidth=1.5)


        
# Final plotting settings and save

#pl.loglog()

ax_main.set_xscale('log')
ax_main.set_yscale('log')

ax_main.tick_params(direction='in', color='k', which='both', top=True, bottom=True, left=True, right=True, labelsize=ticksize)
ax_main.set_xlim(1e-2,2)
ax_main.set_ylim(1e-3, 3e2)
ax_main.set_ylabel(r'$\Delta^2$ (mK$^2$)', fontsize=fontsize, family=font_name)
ax_diff.set_xlabel(r'$k$ (h Mpc$^{-1}$)', fontsize=fontsize, family=font_name)
ax_main.set_title('EoR simulations, 15 Observations')


# Tick formatting
ax_main.set_yticks([1e-2,1e-1,1e0,1e1,1e2])
ax_main.set_yticklabels([r'$10^{-1}$',r'$10^{-1}$', r'$10^{0}$', r'$10^{1}$', r'$10^{2}$'], fontname=font_name)
ax_main.set_xticks([])

a = ax_main
a.yaxis.set_minor_locator(ticker.LogLocator(base=10, numticks=20))
a.yaxis.set_minor_formatter(ticker.NullFormatter())


# --- Bottom panel styling ---
ax_diff.axhline(0.0, color='k', lw=1, alpha=0.5)  # zero reference line
ax_diff.tick_params(direction='in', color='k', which='both', top=False, bottom=True, left=True, right=True, labelsize=ticksize)
ax_diff.set_ylabel(r'Fractional difference', fontsize=fontsize, family=font_name)
ax_diff.set_xticks([1e-2,1e-1,1e0])
ax_diff.set_xticklabels([r'$10^{-2}$', r'$10^{-1}$', r'$10^{0}$'], fontname=font_name)
ax_diff.set_yticks([-1,0,1])
ax_diff.set_yticklabels([r'-1',r'0', r'1'], fontname=font_name)


# Legend
f1_label = ['No w-correction, HEALPix','W-stacking, HEALPix','AW-projection, HEALPix','No w-correction, gridded','W-stacking, gridded','AW-projection, gridded']
patches = [ pl.plot([],[], mec=None, color=colors[type_i], linestyle = linestyles[type_i],
            label="{:s}".format(f1_label[type_i]) )[0]  for type_i in range(len(f1_label)) ]
# legend1 = pl.legend(handles=patches, bbox_to_anchor=(0.72, 0.25),loc='center', ncol=1, frameon=False,
#                     fontsize=legendsize,title_fontsize=legendsize,prop=font)
# #legend1.set_title(title="Derivative matched\nto control cut percent", prop=font_bold)
# #legend1._legend_box.align='left'
# legend1._legend_box.align='left'
# a.add_artist(legend1)


legend1 = ax_main.legend(handles=patches, bbox_to_anchor=(0.72, 0.25), loc='center',
                         ncol=1, frameon=False, fontsize=legendsize, title_fontsize=legendsize, prop=font)
legend1._legend_box.align='left'



pl.savefig(data_dir+'EoR.pdf', bbox_inches='tight', dpi=128)
pl.close('all')

In [75]:
#k0 plotting

# Spectral custom colors
ColorConverter.colors['e']=(94/256, 79/256, 162/256)
ColorConverter.colors['g']=(50/256, 136/256, 189/256)
ColorConverter.colors['h']=(102/256, 194/256, 165/256)
ColorConverter.colors['i']=(253/256, 174/256, 97/256)
ColorConverter.colors['j']=(219/256, 55/256, 82/256)

data_dir = '/Users/nicholebarry/MWA/data/k0_LoBES_compare/'
data_base  = ['1088285600_gridded_uvf__even_odd_joint_fullimg_dirty_yy_dft_averemove_swbh_dencorr_k0power.txt',
              '1088285600_gridded_uvf__even_odd_joint_fullimg_res_yy_dft_averemove_swbh_dencorr_k0power.txt',
              '1088285600_gridded_uvf__even_odd_joint_fullimg_dirty_yy_dft_averemove_swbh_dencorr_k0power_LoBES_extraremoved.txt',
              '1088285600_gridded_uvf__even_odd_joint_fullimg_res_yy_dft_averemove_swbh_dencorr_k0power_LoBES_extraremoved.txt']
data_col = [[0,1],[0,1],[0,1],[0,1]]

# Plot styles per data set
colors=['j','i','h','g','e']

fontsize=16
ticksize=16
legendsize=16

# Initialise the plot
pl.figure(figsize=(8,6))

for file_i, file_name in enumerate(data_base):
    #
    # Load data and select
    data = np.loadtxt(data_dir+file_name,skiprows=1)
    k_edges = data[:,(data_col[file_i])[0]] # Lower and upper edge of bin, (h Mpc^-1)
    power = data[1:,(data_col[file_i])[1]] # Observed power in each bin, (mK^2 h^-3 Mpc^3), first entry is a placeholder
        
    # Construct k center from the edges
    n_k_edges = len(k_edges)
    k_center = (k_edges[1:n_k_edges]+k_edges[0:n_k_edges-1])/2.
        
    # Create histogram style data set by repeating the data 
    # Duplicate the bin edges so each bin has left and right data point
    # Duplicate the power so each bin edge has a measurement
    k_edges_hist  = np.repeat(k_edges,2)[1:-1]
    power_hist  = np.repeat(power,2)
    #if file_i == 1:
    #    delta_hist *= 25.
        
    # Plot the data on the initialised figure
    #if (file_i == 1) or (file_i == 3):
    #    pl.plot(k_edges_hist, power_hist/power_comp, ls='solid', color=colors[file_i],linewidth=2)
    #if (file_i == 0) or (file_i == 2):
    #    power_comp = power_hist
    pl.plot(k_edges_hist, power_hist, ls='solid', color=colors[file_i],linewidth=2)
# Final plotting settings and save

pl.loglog()
pl.tick_params(direction='in', color='k', which='both', top=True, bottom=True, left=True, right=True, labelsize=ticksize)
pl.xlim(1e-2,1e-1)
pl.ylim(5e11, 2e15)
#pl.ylim(1e-2, 1e0)
pl.ylabel(r'k0 power (mK$^2$ h$^{-3}$ Mpc$^3$)', fontsize=fontsize, family=font_name)
pl.xlabel(r'$k_{\perp}$ (h Mpc$^{-1}$)', fontsize=fontsize, family=font_name)

# Tick formatting
#pl.yticks([1e-0,1e3,1e6],fontname = font_name)
pl.xticks(fontname = font_name)
a=pl.gca()
a.yaxis.set_minor_locator(ticker.LogLocator(base=10, numticks=20))
a.yaxis.set_minor_formatter(ticker.NullFormatter())

# Legend
f1_label = ['Calibrated data, GLEAM', 'Residual data, GLEAM', 'Calibrated data, LoBES', 'Residual data, LoBES']
#f1_label = ['Ratio of input to remaining power, GLEAM', 'Ratio of input to remaining power, LoBES']
patches = [ pl.plot([],[], mec=None, color=colors[type_i], 
            label="{:s}".format(f1_label[type_i]) )[0]  for type_i in range(len(f1_label)) ]
legend1 = pl.legend(handles=patches, bbox_to_anchor=(0.22, 0.15),loc='center', ncol=1, frameon=False,
                    fontsize=legendsize,title_fontsize=legendsize,prop=font)
#legend1.set_title(title="Derivative matched\nto control cut percent", prop=font_bold)
legend1._legend_box.align='left'
a.add_artist(legend1)


pl.savefig(data_dir+'Power_Spectrum_Plot.pdf', bbox_inches='tight', dpi=128)
pl.close('all')



FileNotFoundError: /Users/nicholebarry/MWA/data/k0_LoBES_compare/1088285600_gridded_uvf__even_odd_joint_fullimg_dirty_yy_dft_averemove_swbh_dencorr_k0power.txt not found.

<Figure size 800x600 with 0 Axes>